# Idiomatic Pandas: There's a Method Chain to the Madness
- https://tomaugspurger.net/posts/method-chaining/ - Pandas maintainer
- https://datapythonista.me/blog/pandas-the-two-cultures - Pandas maintainer

In [1]:
import pandas as pd

## I. Don't use `inplace`
- using `inplace` doesn't mean a copy won't be made
- `inplace` doesn't work with method chaining
- maintainers don't recommend it's use
- it's planned to be deprecated see [PDEP-8](https://github.com/pandas-dev/pandas/pull/51466)
- There's no longer a benefit because with Copy-on-Write, all the methods already returns views if possible.  (Pandas >= 2.0)

## II. Method Chaining

### Old School

In [18]:
df = pd.read_csv("https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv")

df = df[df.Age < df.Age.quantile(0.99)]

df.Sex.replace({"female": 1, "male": 2}, inplace=True)

df.Age = pd.cut(
    df.Age, 
    bins=[df.Age.min(), 18, 30, 55, df.Age.max()],
    labels=["child", "young adult", "adult", "senior"]
)

df = df.pivot_table(values="Sex", columns="Pclass", index="Age", aggfunc="mean")
df = df.rename_axis("", axis="columns")
df = df.rename("Class {}".format, axis="columns")
df.style.format("{:.2%}")

811 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Idiomatic - Method Chaining
- Pandas maintainer [says method chaining is the preferred way](https://youtu.be/hK6o_TDXXN8?t=371)

In [19]:
df = (
    pd.read_csv("https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv")
    .query("Age < Age.quantile(0.99)")
    .assign(
        Sex=lambda df: df.Sex.replace({"female": 1, "male": 0}),
        Age=lambda df: pd.cut(
            df.Age, 
            bins=[df.Age.min(), 18, 30, 55, df.Age.max()],
            labels=["child", "young adult", "adult", "senior"]
        )
    )
    .pivot_table(values="Sex", columns="Pclass", index="Age", aggfunc="mean")
    .rename_axis("", axis="columns")
    .rename("Class {}".format, axis="columns")
    .style.format("{:.2%}")
)

805 ms ± 26.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Arguments against:
- harder to read.  This is subjective but unless you've given a good faith effort to try using method chaining, you haven't really used it.  Method chaining is as close to R's dplyr piping as we can get.
- debugging is harder
  - can use `.pipe(lambda df: (df, pdb.set_trace())[0])` to set a trace in the middle of a chain
- only works for some functions.  This argument has largely become obsolete with the addition of `.query()` and `.pipe()` where the latter can be pass just about any function pointer that accepts a dataframe.

Arguments for:
- easier to read
  - less keystrokes
  - reads like a pipeline of commands or a recipe
- eliminates intermediate variables
  - intermediate variables are code smell.  
  - avoids `SettingWithCopy` warning
- improved performance
  - intermediate variables use more memory